Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (190000, 28, 28) (190000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (190000, 784) (190000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
batch_size = 128
hidden_nodes_layer_1 = 1024
learning_rate = .2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_regularization_ammount = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_layer_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1]))
  biases_layer_1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
    
  
  weights_layer_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer_1, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  ouput_1 = tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, weights_layer_1), biases_layer_1))

  logits = tf.add(tf.matmul(ouput_1,weights_layer_2),biases_layer_2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels))+ tf_regularization_ammount*tf.nn.l2_loss(weights_layer_1)+ tf_regularization_ammount*tf.nn.l2_loss(weights_layer_2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.add(
    tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights_layer_1), biases_layer_1)),weights_layer_2),biases_layer_2))

  test_prediction = tf.nn.softmax(tf.add(
    tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights_layer_1), biases_layer_1)),weights_layer_2),biases_layer_2))

In [11]:
num_steps = 1001
regularization_amount = 0.002

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    indexes = np.random.choice(train_labels.shape[0], batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[indexes, :]
    batch_labels = train_labels[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_regularization_ammount : regularization_amount}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1074.986694
Minibatch accuracy: 7.0%
Validation accuracy: 28.5%
Minibatch loss at step 100: 601.101135
Minibatch accuracy: 82.8%
Validation accuracy: 77.8%
Minibatch loss at step 200: 568.395264
Minibatch accuracy: 73.4%
Validation accuracy: 78.5%
Minibatch loss at step 300: 504.168182
Minibatch accuracy: 86.7%
Validation accuracy: 77.7%
Minibatch loss at step 400: 464.737579
Minibatch accuracy: 78.9%
Validation accuracy: 78.3%
Minibatch loss at step 500: 431.016632
Minibatch accuracy: 73.4%
Validation accuracy: 80.9%
Minibatch loss at step 600: 391.653839
Minibatch accuracy: 85.2%
Validation accuracy: 81.3%
Minibatch loss at step 700: 359.942322
Minibatch accuracy: 87.5%
Validation accuracy: 82.2%
Minibatch loss at step 800: 333.887939
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%
Minibatch loss at step 900: 308.124207
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 281.778503
Minibatch accuracy: 84.4%
Vali

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

* If we keep the same ammount of regularization (.002) test performance decreases to ~77%.
* If we increase the ammount of regularization (.1) performance seems to be better ~85%

 * We also see that training data accuracy reach 100% with small regularization; however validation accuracy keeps smaller. The NN is overfitting.

With greater regularization value training and validation accuracy tend to be more similar.

In [12]:
batch_size = 128
hidden_nodes_layer_1 = 1024
learning_rate = .2
num_steps = 1001
regularization_amount = 0.1

size_truncated_dataset = 1000
indexes = np.random.choice(train_labels.shape[0],size_truncated_dataset)
train_dataset_truncated = train_dataset[indexes, :]
train_labels_truncated = train_labels[indexes, :]
print('Training set truncated:', train_dataset_truncated.shape, train_labels_truncated.shape)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_regularization_ammount = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_layer_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1]))
  biases_layer_1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
    
  
  weights_layer_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer_1, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  ouput_1 = tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, weights_layer_1), biases_layer_1))
  logits = tf.matmul(ouput_1,weights_layer_2)+biases_layer_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels))+ tf_regularization_ammount*tf.nn.l2_loss(weights_layer_1)+ tf_regularization_ammount*tf.nn.l2_loss(weights_layer_2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)

  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)



with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    indexes = np.random.choice(train_labels_truncated.shape[0], batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_truncated[indexes, :]
    batch_labels = train_labels_truncated[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_regularization_ammount : regularization_amount}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Training set truncated: (1000, 784) (1000, 10)
Initialized
Minibatch loss at step 0: 31921.125000
Minibatch accuracy: 6.2%
Validation accuracy: 23.1%
Minibatch loss at step 100: 552.096558
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 200: 10.996307
Minibatch accuracy: 91.4%
Validation accuracy: 78.9%
Minibatch loss at step 300: 1.693183
Minibatch accuracy: 80.5%
Validation accuracy: 78.6%
Minibatch loss at step 400: 1.352038
Minibatch accuracy: 83.6%
Validation accuracy: 78.3%
Minibatch loss at step 500: 1.305722
Minibatch accuracy: 86.7%
Validation accuracy: 78.8%
Minibatch loss at step 600: 1.373650
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 700: 1.414299
Minibatch accuracy: 78.9%
Validation accuracy: 78.8%
Minibatch loss at step 800: 1.275034
Minibatch accuracy: 83.6%
Validation accuracy: 79.1%
Minibatch loss at step 900: 1.297622
Minibatch accuracy: 86.7%
Validation accuracy: 79.1%
Minibatch loss at step 1000: 1.44231

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

With `keep_prob_dropout` probability ~10% we significantly improve the accuracy: (88%).

We also see similar validation-training error thus we conclude overfitting is not taking place.

It is also curious that validation error accuracy reaches ~ 80% around step 300: whereas training accuracy is smaller. I think this is due to that training prediction is only taking into account the 10% active hidden neurons at this step; whereas valid prediction takes into account all hidden units.

In [13]:
batch_size = 128
hidden_nodes_layer_1 = 1024
learning_rate = .02
num_steps = 1001
keep_prob_dropout = 0.1
size_truncated_dataset = 1000

indexes = np.random.choice(train_labels.shape[0],size_truncated_dataset)
train_dataset_truncated = train_dataset[indexes, :]
train_labels_truncated = train_labels[indexes, :]

print('Training set truncated:', train_dataset_truncated.shape, train_labels_truncated.shape)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_regularization_ammount = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_layer_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1]))
  biases_layer_1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
    
  
  weights_layer_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer_1, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  ouput_1 = tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, weights_layer_1), biases_layer_1))
  tf_keep_prob_dropout = tf.placeholder("float")
  ouput_1_drop = tf.nn.dropout(ouput_1, tf_keep_prob_dropout)
  
  logits = tf.matmul(ouput_1_drop,weights_layer_2)+biases_layer_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)

  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    indexes = np.random.choice(train_labels_truncated.shape[0], batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_truncated[indexes, :]
    batch_labels = train_labels_truncated[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_keep_prob_dropout : keep_prob_dropout}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Training set truncated: (1000, 784) (1000, 10)
Initialized
Minibatch loss at step 0: 1105.660156
Minibatch accuracy: 13.3%
Validation accuracy: 10.8%
Minibatch loss at step 100: 233.564972
Minibatch accuracy: 60.2%
Validation accuracy: 76.8%
Minibatch loss at step 200: 151.751648
Minibatch accuracy: 65.6%
Validation accuracy: 79.2%
Minibatch loss at step 300: 104.093155
Minibatch accuracy: 68.0%
Validation accuracy: 79.4%
Minibatch loss at step 400: 137.980057
Minibatch accuracy: 66.4%
Validation accuracy: 79.5%
Minibatch loss at step 500: 84.306366
Minibatch accuracy: 72.7%
Validation accuracy: 80.0%
Minibatch loss at step 600: 65.246048
Minibatch accuracy: 78.9%
Validation accuracy: 80.1%
Minibatch loss at step 700: 48.921516
Minibatch accuracy: 78.9%
Validation accuracy: 80.1%
Minibatch loss at step 800: 35.975128
Minibatch accuracy: 79.7%
Validation accuracy: 80.0%
Minibatch loss at step 900: 35.682419
Minibatch accuracy: 79.7%
Validation accuracy: 80.3%
Minibatch loss at step 1000

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [24]:
batch_size = 128
hidden_nodes_layer_1 = 1024
hidden_nodes_layer_2 = 2048
learning_rate = .02
num_steps = 4000
keep_prob_dropout = 0.85


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_regularization_ammount = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  def multilayer_perceptron(_X, _weights, _biases,dropout=0):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) #Hidden layer with RELU activation
    if dropout:
        layer_1 = tf.nn.dropout(layer_1, keep_prob_dropout)
    
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) #Hidden layer with RELU activation
    if dropout:
        layer_2 = tf.nn.dropout(layer_2, keep_prob_dropout)
    return tf.add(tf.matmul(layer_2, _weights['out']), _biases['out'])

  # Store layers weight & bias
  weights = {
      'h1': tf.Variable(tf.random_normal([image_size * image_size, hidden_nodes_layer_1])),
      'h2': tf.Variable(tf.random_normal([hidden_nodes_layer_1, hidden_nodes_layer_2])),
      'out': tf.Variable(tf.random_normal([hidden_nodes_layer_2, num_labels]))
  }
  biases = {
    'b1': tf.Variable(tf.random_normal([hidden_nodes_layer_1])),
    'b2': tf.Variable(tf.random_normal([hidden_nodes_layer_2])),
    'out': tf.Variable(tf.random_normal([num_labels]))
  }
  
  
  logits = multilayer_perceptron(tf_train_dataset, weights, biases,1)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels))
  
  # Optimizer.
#  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                             30, learning_rate_decay, staircase=True)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    multilayer_perceptron(tf_valid_dataset, weights, biases))

  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights, biases))


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    indexes = np.random.choice(train_labels.shape[0], batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[indexes, :]
    batch_labels = train_labels[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14815.375000
Minibatch accuracy: 12.5%
Validation accuracy: 30.1%
Minibatch loss at step 100: 2735.343262
Minibatch accuracy: 75.8%
Validation accuracy: 81.0%
Minibatch loss at step 200: 1772.255981
Minibatch accuracy: 73.4%
Validation accuracy: 82.2%
Minibatch loss at step 300: 1113.777344
Minibatch accuracy: 78.9%
Validation accuracy: 82.7%
Minibatch loss at step 400: 1451.413086
Minibatch accuracy: 78.1%
Validation accuracy: 83.1%
Minibatch loss at step 500: 1511.902588
Minibatch accuracy: 80.5%
Validation accuracy: 83.4%
Minibatch loss at step 600: 732.967773
Minibatch accuracy: 86.7%
Validation accuracy: 83.6%
Minibatch loss at step 700: 1260.224854
Minibatch accuracy: 71.9%
Validation accuracy: 83.8%
Minibatch loss at step 800: 937.977173
Minibatch accuracy: 80.5%
Validation accuracy: 84.0%
Minibatch loss at step 900: 857.971313
Minibatch accuracy: 77.3%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 811.334595
Minibatch accuracy: 80

In [36]:
batch_size = 128
hidden_nodes_layer_1 = 1024
hidden_nodes_layer_2 = num_labels
starter_learning_rate = .5
num_steps = 3001
keep_prob_dropout = 0.5
learning_rate_decay = .96
print('Training set :', train_dataset.shape, train_labels.shape)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_layer_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1]))
  biases_layer_1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
    
  
  weights_layer_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer_1, hidden_nodes_layer_2]))
  biases_layer_2 = tf.Variable(tf.zeros([hidden_nodes_layer_2]))
  
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer_2, num_labels]))
  
  biases_out = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  ouput_1 = tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, weights_layer_1), biases_layer_1))
  #output_1 = tf.contrib.layers.fully_connected(
  #  tf_train_dataset,hidden_nodes_layer_1,tf.nn.relu)
  
  ouput_1_drop = tf.nn.dropout(ouput_1, keep_prob_dropout)
  
  ouput_2 = tf.nn.relu(tf.add(tf.matmul(ouput_1_drop, weights_layer_2), biases_layer_2))
  ouput_2_drop = tf.nn.dropout(ouput_2, keep_prob_dropout)

  logits = tf.add(tf.matmul(ouput_2_drop,weights_out),biases_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels))
  
  # Optimizer.
  step = tf.Variable(0, trainable=False)

  learning_rate = tf.train.exponential_decay(starter_learning_rate, step,
                                             30, learning_rate_decay, staircase=True)
  
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss,global_step=step)
  step = step+1
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(
    tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights_layer_1), biases_layer_1)),weights_layer_2),
                    biases_layer_2)),weights_out),biases_out))

  test_prediction =   tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(
    tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights_layer_1), biases_layer_1)),weights_layer_2),
                    biases_layer_2)),weights_out),biases_out))

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    indexes = np.random.choice(train_labels.shape[0], batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[indexes, :]
    batch_labels = train_labels[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      current_accuracy = accuracy(
        valid_prediction.eval(), valid_labels)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % current_accuracy)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Training set : (190000, 784) (190000, 10)
Initialized
Minibatch loss at step 0: 799.505554
Minibatch accuracy: 9.4%
Validation accuracy: 11.4%
Minibatch loss at step 100: 2.306981
Minibatch accuracy: 11.7%
Validation accuracy: 9.9%
Minibatch loss at step 200: 2.304481
Minibatch accuracy: 10.2%
Validation accuracy: 9.9%
Minibatch loss at step 300: 2.305083
Minibatch accuracy: 7.0%
Validation accuracy: 10.5%
Minibatch loss at step 400: 2.303695
Minibatch accuracy: 10.9%
Validation accuracy: 9.7%
Minibatch loss at step 500: 2.303989
Minibatch accuracy: 9.4%
Validation accuracy: 9.7%
Minibatch loss at step 600: 2.297965
Minibatch accuracy: 18.0%
Validation accuracy: 9.9%
Minibatch loss at step 700: 2.302177
Minibatch accuracy: 10.2%
Validation accuracy: 9.7%
Minibatch loss at step 800: 2.301372
Minibatch accuracy: 10.9%
Validation accuracy: 9.9%
Minibatch loss at step 900: 2.301842
Minibatch accuracy: 17.2%
Validation accuracy: 9.9%
Minibatch loss at step 1000: 2.303378
Minibatch accuracy: